In [1]:
import numpy as np
import csv
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
import pandas as pd
from PIL import Image, ImageOps
import h5py
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block
    
    Arguments:
    X: Input
    f: integer, specifying the shape of the middle CONV's window for the main path
    filters: Number of filters in the CONV layers of the main path
    stage: Integer, used to name the layers, depending on their position in the network
    block: String, used to name the layers, depending on their position in the network
    
    Returns:
    X: Output of the identity block
    """
    # Defining some name basis
    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base = "bn" + str(stage) + block + "_branch"
    
    # Retrieve filters
    F1, F2, F3, F4 = filters
    
    # Save the input value, to use it later to add to the main path
    X_shortcut = X
    
    # first component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1, 1), padding = "valid", name = conv_name_base + "2a",
               kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2a")(X)
    X = Activation("relu")(X)
    
    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = "same", name = conv_name_base + "2b",
              kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2b")(X)
    X = Activation("relu")(X)
    
    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), padding = "valid", name = conv_name_base + "2c",
              kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2c")(X)
    X = Activation("relu")(X)
    
    # Fourth component of the main path
    X = Conv2D(filters = F4, kernel_size = (f, f), strides = (1, 1), padding = "same", name = conv_name_base + "2d",
              kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2d")(X)
    X = Activation("relu")(X)
    
    # Add shortcut to the main path and pass it through a RelU activation
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)
    
    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block
    
    Arguments:
    X: Input
    f: Integer, specifying the shape of the middle CONV's window for the main path
    filters: filters: Number of filters in the CONV layers of the main path
    stage: Integer, used to name the layers, depending on their position in the network
    block: String, used to name the layers, depending on their position in the network
    s: Integer, specifying the stride to be used
    
    Returns:
    X: Output of the convolutional block
    """
    
    # Defining name basis:
    conv_name_base =  "res" + str(stage) + block + "_branch"
    bn_name_base = "bn" + str(stage) + block + "_branch"
    
    # Retrieve filters
    F1, F2, F3, F4 = filters
    
    # Save the input value
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (s, s), padding = "valid", name = conv_name_base + "2a",
              kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2a")(X)
    X = Activation("relu")(X)
    
    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = "same", name = conv_name_base + "2b",
              kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2b")(X)
    X = Activation("relu")(X)
    
    #Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s, s), padding = "valid", name = conv_name_base + "2c",
              kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2c")(X)
    X = Activation("relu")(X)
    
    #Fourth component of main path
    X = Conv2D(filters = F4, kernel_size = (f, f), strides = (1, 1), padding = "same", name = conv_name_base + "2d",
               kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2d")(X)
    X = Activation("relu")(X)
    
    # Shortcut path
    X_shortcut = Conv2D(filters = F4, kernel_size = (f, f), strides = (1, 1), padding = "same", name = conv_name_base + "1",
                       kernel_initializer = glorot_uniform(seed = 0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name =  bn_name_base + "1")(X_shortcut)
    
    # Add shortcut to the main path and pass it through a RelU activation
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)
    
    return X

In [ ]:
def ResNet(input_shape = (512, 512, 3), classes = 1):
    """
    Implementation of a Residual Network with 150 layers
    
    Inputs:
    input_shape: shape of the images
    classes: Integer, number of classes
    
    Returns:
    model: a model() instance in Keras
    """
    
    # Define the input as a tensor
    X_input = Input(input_shape)
    
    # Zero padding
    X = ZeroPadding2D((3, 3))(X_input) 
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = "conv1", kernel_initializer=  glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = "bn_conv1")(X)
    X = Activation("relu")(X)
    X = MaxPooling2D((3, 3), strides = (2, 2))(X)
    
    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256, 256], stage = 2, block = "a", s = 1)
    X = identity_block(X, 3, [64, 64, 256, 256], stage = 2, block = "b")
    X = identity_block(X, 3, [64, 64, 256, 256], stage = 2, block = "c")
    
    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512, 512], stage = 3, block = "a", s = 1)
    X = identity_block(X, 3, [128, 128, 512, 512], stage = 3, block = "b")
    X = identity_block(X, 3, [128, 128, 512, 512], stage = 3, block = "c")
    X = identity_block(X, 3, [128, 128, 512, 512], stage = 3, block = "d")
    
    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024, 1024], stage = 4, block = "a", s = 1)
    X = identity_block(X, 3, [256, 256, 1024, 1024], stage = 4, block = "b")
    X = identity_block(X, 3, [256, 256, 1024, 1024], stage = 4, block = "c")
    X = identity_block(X, 3, [256, 256, 1024, 1024], stage = 4, block = "d")
    X = identity_block(X, 3, [256, 256, 1024, 1024], stage = 4, block = "e")
    
    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048, 2048], stage = 5, block = "a", s = 1)
    X = identity_block(X, 3, [512, 512, 2048, 2048], stage = 5, block = "b")
    X = identity_block(X, 3, [512, 512, 2048, 2048], stage = 5, block = "c")
    X = identity_block(X, 3, [512, 512, 2048, 2048], stage = 5, block = "d")
    X = identity_block(X, 3, [512, 512, 2048, 2048], stage = 5, block = "e")
    X = identity_block(X, 3, [512, 512, 2048, 2048], stage = 5, block = "f")
    
    # Stage 6
    X = convolutional_block(X, f = 3, filters = [1024, 1024, 4096, 4096], stage = 6, block = "a", s = 1)
    X = identity_block(X, 3, [1024, 1024, 4096, 4096], stage = 6, block = "b")
    X = identity_block(X, 3, [1024, 1024, 4096, 4096], stage = 6, block = "c")
    X = identity_block(X, 3, [1024, 1024, 4096, 4096], stage = 6, block = "d")
    
    # Average Pool
    X = AveragePooling2D(pool_size = (2, 2), padding = "same")(X)
    
    # Output layer
    X = Flatten()(X)
    X = Dense(classes, activation = "sigmoid", name = "fc" + str(classes), kernel_initializer = glorot_uniform(seed = 0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name = "ResNet")
    
    return model    

In [ ]:
model = ResNet(input_shape = (512, 512, 3), classes = 1)

In [ ]:
model.compile(optimizer='adam', loss='logcosh', metrics=['accuracy'])

In [2]:
train_data_x = pd.read_csv("MURA-v1.1/train_image_paths.csv")
train_data_y = pd.read_csv("MURA-v1.1/train_labeled_studies.csv")
valid_data_x = pd.read_csv("MURA-v1.1/valid_image_paths.csv")
valid_data_y = pd.read_csv("MURA-v1.1/valid_labeled_studies.csv")
train_x_orig = train_data_x.iloc[:, 0]
train_y_orig = train_data_y.iloc[:, 1]
valid_x_orig = valid_data_x.iloc[:, 0]
valid_y_orig = valid_data_y.iloc[:, 1]

In [ ]:
train_data_array = []
desired_size = 512

for i in range (len(train_x_orig)):
    # Convet images from grayscale to RGB
    image = Image.open(train_x_orig.iloc[i])
    image = image.convert("RGB")
    
    # Resize image so every image got the same shape
    old_size = image.size  # old_size[0] is in (width, height) format
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    
    # Use thumbnail() or resize() method to resize the input image  
    image = image.resize(new_size, Image.ANTIALIAS)
    
    # Create a new image and paste the resized on it
    new_image = Image.new("RGB", (desired_size, desired_size))
    new_image.paste(image, ((desired_size-new_size[0])//2,
                    (desired_size-new_size[1])//2))

    array = np.array(new_image)
    
    #Test the dimensions of the data
    print("Shape = " + str(array.shape) + " " + str(i))
    
    train_data_array.append(array)

In [3]:
# We do it twice, once for training data and another one for validation data
valid_data_array = []
desired_size = 512

for i in range (len(valid_x_orig)):
    image = Image.open(valid_x_orig.iloc[i])
    image = image.convert("RGB")
    
    old_size = image.size  
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
   
    image = image.resize(new_size, Image.ANTIALIAS)
    
    new_image = Image.new("RGB", (desired_size, desired_size))
    new_image.paste(image, ((desired_size-new_size[0])//2,
                    (desired_size-new_size[1])//2))

    array = np.array(new_image)
    
    print("Shape = " + str(array.shape) + " " + str(i))
    
    valid_data_array.append(array)

Shape = (512, 512, 3) 0
Shape = (512, 512, 3) 1
Shape = (512, 512, 3) 2
Shape = (512, 512, 3) 3
Shape = (512, 512, 3) 4
Shape = (512, 512, 3) 5
Shape = (512, 512, 3) 6
Shape = (512, 512, 3) 7
Shape = (512, 512, 3) 8
Shape = (512, 512, 3) 9
Shape = (512, 512, 3) 10
Shape = (512, 512, 3) 11
Shape = (512, 512, 3) 12
Shape = (512, 512, 3) 13
Shape = (512, 512, 3) 14
Shape = (512, 512, 3) 15
Shape = (512, 512, 3) 16
Shape = (512, 512, 3) 17
Shape = (512, 512, 3) 18
Shape = (512, 512, 3) 19
Shape = (512, 512, 3) 20
Shape = (512, 512, 3) 21
Shape = (512, 512, 3) 22
Shape = (512, 512, 3) 23
Shape = (512, 512, 3) 24
Shape = (512, 512, 3) 25
Shape = (512, 512, 3) 26
Shape = (512, 512, 3) 27
Shape = (512, 512, 3) 28
Shape = (512, 512, 3) 29
Shape = (512, 512, 3) 30
Shape = (512, 512, 3) 31
Shape = (512, 512, 3) 32
Shape = (512, 512, 3) 33
Shape = (512, 512, 3) 34
Shape = (512, 512, 3) 35
Shape = (512, 512, 3) 36
Shape = (512, 512, 3) 37
Shape = (512, 512, 3) 38
Shape = (512, 512, 3) 39
Shape = (5

Shape = (512, 512, 3) 334
Shape = (512, 512, 3) 335
Shape = (512, 512, 3) 336
Shape = (512, 512, 3) 337
Shape = (512, 512, 3) 338
Shape = (512, 512, 3) 339
Shape = (512, 512, 3) 340
Shape = (512, 512, 3) 341
Shape = (512, 512, 3) 342
Shape = (512, 512, 3) 343
Shape = (512, 512, 3) 344
Shape = (512, 512, 3) 345
Shape = (512, 512, 3) 346
Shape = (512, 512, 3) 347
Shape = (512, 512, 3) 348
Shape = (512, 512, 3) 349
Shape = (512, 512, 3) 350
Shape = (512, 512, 3) 351
Shape = (512, 512, 3) 352
Shape = (512, 512, 3) 353
Shape = (512, 512, 3) 354
Shape = (512, 512, 3) 355
Shape = (512, 512, 3) 356
Shape = (512, 512, 3) 357
Shape = (512, 512, 3) 358
Shape = (512, 512, 3) 359
Shape = (512, 512, 3) 360
Shape = (512, 512, 3) 361
Shape = (512, 512, 3) 362
Shape = (512, 512, 3) 363
Shape = (512, 512, 3) 364
Shape = (512, 512, 3) 365
Shape = (512, 512, 3) 366
Shape = (512, 512, 3) 367
Shape = (512, 512, 3) 368
Shape = (512, 512, 3) 369
Shape = (512, 512, 3) 370
Shape = (512, 512, 3) 371
Shape = (512

Shape = (512, 512, 3) 655
Shape = (512, 512, 3) 656
Shape = (512, 512, 3) 657
Shape = (512, 512, 3) 658
Shape = (512, 512, 3) 659
Shape = (512, 512, 3) 660
Shape = (512, 512, 3) 661
Shape = (512, 512, 3) 662
Shape = (512, 512, 3) 663
Shape = (512, 512, 3) 664
Shape = (512, 512, 3) 665
Shape = (512, 512, 3) 666
Shape = (512, 512, 3) 667
Shape = (512, 512, 3) 668
Shape = (512, 512, 3) 669
Shape = (512, 512, 3) 670
Shape = (512, 512, 3) 671
Shape = (512, 512, 3) 672
Shape = (512, 512, 3) 673
Shape = (512, 512, 3) 674
Shape = (512, 512, 3) 675
Shape = (512, 512, 3) 676
Shape = (512, 512, 3) 677
Shape = (512, 512, 3) 678
Shape = (512, 512, 3) 679
Shape = (512, 512, 3) 680
Shape = (512, 512, 3) 681
Shape = (512, 512, 3) 682
Shape = (512, 512, 3) 683
Shape = (512, 512, 3) 684
Shape = (512, 512, 3) 685
Shape = (512, 512, 3) 686
Shape = (512, 512, 3) 687
Shape = (512, 512, 3) 688
Shape = (512, 512, 3) 689
Shape = (512, 512, 3) 690
Shape = (512, 512, 3) 691
Shape = (512, 512, 3) 692
Shape = (512

Shape = (512, 512, 3) 978
Shape = (512, 512, 3) 979
Shape = (512, 512, 3) 980
Shape = (512, 512, 3) 981
Shape = (512, 512, 3) 982
Shape = (512, 512, 3) 983
Shape = (512, 512, 3) 984
Shape = (512, 512, 3) 985
Shape = (512, 512, 3) 986
Shape = (512, 512, 3) 987
Shape = (512, 512, 3) 988
Shape = (512, 512, 3) 989
Shape = (512, 512, 3) 990
Shape = (512, 512, 3) 991
Shape = (512, 512, 3) 992
Shape = (512, 512, 3) 993
Shape = (512, 512, 3) 994
Shape = (512, 512, 3) 995
Shape = (512, 512, 3) 996
Shape = (512, 512, 3) 997
Shape = (512, 512, 3) 998
Shape = (512, 512, 3) 999
Shape = (512, 512, 3) 1000
Shape = (512, 512, 3) 1001
Shape = (512, 512, 3) 1002
Shape = (512, 512, 3) 1003
Shape = (512, 512, 3) 1004
Shape = (512, 512, 3) 1005
Shape = (512, 512, 3) 1006
Shape = (512, 512, 3) 1007
Shape = (512, 512, 3) 1008
Shape = (512, 512, 3) 1009
Shape = (512, 512, 3) 1010
Shape = (512, 512, 3) 1011
Shape = (512, 512, 3) 1012
Shape = (512, 512, 3) 1013
Shape = (512, 512, 3) 1014
Shape = (512, 512, 3) 1

Shape = (512, 512, 3) 1299
Shape = (512, 512, 3) 1300
Shape = (512, 512, 3) 1301
Shape = (512, 512, 3) 1302
Shape = (512, 512, 3) 1303
Shape = (512, 512, 3) 1304
Shape = (512, 512, 3) 1305
Shape = (512, 512, 3) 1306
Shape = (512, 512, 3) 1307
Shape = (512, 512, 3) 1308
Shape = (512, 512, 3) 1309
Shape = (512, 512, 3) 1310
Shape = (512, 512, 3) 1311
Shape = (512, 512, 3) 1312
Shape = (512, 512, 3) 1313
Shape = (512, 512, 3) 1314
Shape = (512, 512, 3) 1315
Shape = (512, 512, 3) 1316
Shape = (512, 512, 3) 1317
Shape = (512, 512, 3) 1318
Shape = (512, 512, 3) 1319
Shape = (512, 512, 3) 1320
Shape = (512, 512, 3) 1321
Shape = (512, 512, 3) 1322
Shape = (512, 512, 3) 1323
Shape = (512, 512, 3) 1324
Shape = (512, 512, 3) 1325
Shape = (512, 512, 3) 1326
Shape = (512, 512, 3) 1327
Shape = (512, 512, 3) 1328
Shape = (512, 512, 3) 1329
Shape = (512, 512, 3) 1330
Shape = (512, 512, 3) 1331
Shape = (512, 512, 3) 1332
Shape = (512, 512, 3) 1333
Shape = (512, 512, 3) 1334
Shape = (512, 512, 3) 1335
S

...

Shape = (512, 512, 3) 1923
Shape = (512, 512, 3) 1924
Shape = (512, 512, 3) 1925
Shape = (512, 512, 3) 1926
Shape = (512, 512, 3) 1927
Shape = (512, 512, 3) 1928
Shape = (512, 512, 3) 1929
Shape = (512, 512, 3) 1930
Shape = (512, 512, 3) 1931
Shape = (512, 512, 3) 1932
Shape = (512, 512, 3) 1933
Shape = (512, 512, 3) 1934
Shape = (512, 512, 3) 1935
Shape = (512, 512, 3) 1936
Shape = (512, 512, 3) 1937
Shape = (512, 512, 3) 1938
Shape = (512, 512, 3) 1939
Shape = (512, 512, 3) 1940
Shape = (512, 512, 3) 1941
Shape = (512, 512, 3) 1942
Shape = (512, 512, 3) 1943
Shape = (512, 512, 3) 1944
Shape = (512, 512, 3) 1945
Shape = (512, 512, 3) 1946
Shape = (512, 512, 3) 1947
Shape = (512, 512, 3) 1948
Shape = (512, 512, 3) 1949
Shape = (512, 512, 3) 1950
Shape = (512, 512, 3) 1951
Shape = (512, 512, 3) 1952
Shape = (512, 512, 3) 1953
Shape = (512, 512, 3) 1954
Shape = (512, 512, 3) 1955
Shape = (512, 512, 3) 1956
Shape = (512, 512, 3) 1957
Shape = (512, 512, 3) 1958
Shape = (512, 512, 3) 1959
S

Shape = (512, 512, 3) 2238
Shape = (512, 512, 3) 2239
Shape = (512, 512, 3) 2240
Shape = (512, 512, 3) 2241
Shape = (512, 512, 3) 2242
Shape = (512, 512, 3) 2243
Shape = (512, 512, 3) 2244
Shape = (512, 512, 3) 2245
Shape = (512, 512, 3) 2246
Shape = (512, 512, 3) 2247
Shape = (512, 512, 3) 2248
Shape = (512, 512, 3) 2249
Shape = (512, 512, 3) 2250
Shape = (512, 512, 3) 2251
Shape = (512, 512, 3) 2252
Shape = (512, 512, 3) 2253
Shape = (512, 512, 3) 2254
Shape = (512, 512, 3) 2255
Shape = (512, 512, 3) 2256
Shape = (512, 512, 3) 2257
Shape = (512, 512, 3) 2258
Shape = (512, 512, 3) 2259
Shape = (512, 512, 3) 2260
Shape = (512, 512, 3) 2261
Shape = (512, 512, 3) 2262
Shape = (512, 512, 3) 2263
Shape = (512, 512, 3) 2264
Shape = (512, 512, 3) 2265
Shape = (512, 512, 3) 2266
Shape = (512, 512, 3) 2267
Shape = (512, 512, 3) 2268
Shape = (512, 512, 3) 2269
Shape = (512, 512, 3) 2270
Shape = (512, 512, 3) 2271
Shape = (512, 512, 3) 2272
Shape = (512, 512, 3) 2273
Shape = (512, 512, 3) 2274
S

Shape = (512, 512, 3) 2554
Shape = (512, 512, 3) 2555
Shape = (512, 512, 3) 2556
Shape = (512, 512, 3) 2557
Shape = (512, 512, 3) 2558
Shape = (512, 512, 3) 2559
Shape = (512, 512, 3) 2560
Shape = (512, 512, 3) 2561
Shape = (512, 512, 3) 2562
Shape = (512, 512, 3) 2563
Shape = (512, 512, 3) 2564
Shape = (512, 512, 3) 2565
Shape = (512, 512, 3) 2566
Shape = (512, 512, 3) 2567
Shape = (512, 512, 3) 2568
Shape = (512, 512, 3) 2569
Shape = (512, 512, 3) 2570
Shape = (512, 512, 3) 2571
Shape = (512, 512, 3) 2572
Shape = (512, 512, 3) 2573
Shape = (512, 512, 3) 2574
Shape = (512, 512, 3) 2575
Shape = (512, 512, 3) 2576
Shape = (512, 512, 3) 2577
Shape = (512, 512, 3) 2578
Shape = (512, 512, 3) 2579
Shape = (512, 512, 3) 2580
Shape = (512, 512, 3) 2581
Shape = (512, 512, 3) 2582
Shape = (512, 512, 3) 2583
Shape = (512, 512, 3) 2584
Shape = (512, 512, 3) 2585
Shape = (512, 512, 3) 2586
Shape = (512, 512, 3) 2587
Shape = (512, 512, 3) 2588
Shape = (512, 512, 3) 2589
Shape = (512, 512, 3) 2590
S

Shape = (512, 512, 3) 2883
Shape = (512, 512, 3) 2884
Shape = (512, 512, 3) 2885
Shape = (512, 512, 3) 2886
Shape = (512, 512, 3) 2887
Shape = (512, 512, 3) 2888
Shape = (512, 512, 3) 2889
Shape = (512, 512, 3) 2890
Shape = (512, 512, 3) 2891
Shape = (512, 512, 3) 2892
Shape = (512, 512, 3) 2893
Shape = (512, 512, 3) 2894
Shape = (512, 512, 3) 2895
Shape = (512, 512, 3) 2896
Shape = (512, 512, 3) 2897
Shape = (512, 512, 3) 2898
Shape = (512, 512, 3) 2899
Shape = (512, 512, 3) 2900
Shape = (512, 512, 3) 2901
Shape = (512, 512, 3) 2902
Shape = (512, 512, 3) 2903
Shape = (512, 512, 3) 2904
Shape = (512, 512, 3) 2905
Shape = (512, 512, 3) 2906
Shape = (512, 512, 3) 2907
Shape = (512, 512, 3) 2908
Shape = (512, 512, 3) 2909
Shape = (512, 512, 3) 2910
Shape = (512, 512, 3) 2911
Shape = (512, 512, 3) 2912
Shape = (512, 512, 3) 2913
Shape = (512, 512, 3) 2914
Shape = (512, 512, 3) 2915
Shape = (512, 512, 3) 2916
Shape = (512, 512, 3) 2917
Shape = (512, 512, 3) 2918
Shape = (512, 512, 3) 2919
S

Shape = (512, 512, 3) 3188
Shape = (512, 512, 3) 3189
Shape = (512, 512, 3) 3190
Shape = (512, 512, 3) 3191
Shape = (512, 512, 3) 3192
Shape = (512, 512, 3) 3193
Shape = (512, 512, 3) 3194
Shape = (512, 512, 3) 3195


In [ ]:
# Stack all the images
train_data = np.stack(train_data_array, axis = 0)
valid_data = np.stack(valid_data_array, axis = 0)

In [ ]:
# Normalize image vectors
X_train = train_data/255
X_valid = valid_data/255
Y_train = train_y_orig
Y_valid = valid_y_orig
print("number of training examples = " + str(X_train.shape[0]))
print("number of test examples = " + str(X_valid.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_valid.shape))
print("Y_test shape: " + str(Y_valid.shape))

In [ ]:
# Train the model

model.fit(X_train, Y_train, epochs = 3, batch_size = 64)

In [ ]:
preds = model.evaluate(X_valid, Y_valid)
print("Loss = " + str(preds[0]))
print("Test Accuracy = " + str(preds[1]))